# Commodities: Price, Returns, and Volatility Snapshot

This notebook provides a baseline for commodities monitoring from the FinUties API.

## What this notebook teaches
- Consistent API loading from `notebooks/.env`
- Price-to-return transformation workflow
- Volatility context using rolling dispersion

In [2]:
from pathlib import Path
import os

import matplotlib.pyplot as plt
import pandas as pd
import requests
from dotenv import load_dotenv

load_dotenv(Path.cwd().parent / ".env")
API_ORIGIN = os.getenv("FINUTIES_API_ORIGIN", "https://data.finuties.com").rstrip("/")
API_KEY = os.getenv("FINUTIES_API_KEY", "").strip()
if not API_KEY:
    raise ValueError("Missing FINUTIES_API_KEY in notebooks/.env")
HEADERS = {"Authorization": f"Bearer {API_KEY}"}

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
response = requests.get(
    f"{API_ORIGIN}/api/v1/data/commodities/prices",
    headers=HEADERS,
    params={"limit": 250},
    timeout=30,
)
response.raise_for_status()
payload = response.json()
rows = payload if isinstance(payload, list) else payload.get("items") or payload.get("data") or []

df = pd.DataFrame(rows)
if not {"date", "price"}.issubset(df.columns):
    raise ValueError(f"Unexpected commodities schema: {list(df.columns)}")

In [ ]:
df["date"] = pd.to_datetime(df["date"], errors="coerce")
df["price"] = pd.to_numeric(df["price"], errors="coerce")
df = df.dropna(subset=["date", "price"]).sort_values("date").reset_index(drop=True)

df["returns"] = df["price"].pct_change().fillna(0.0)
df["rolling_vol_20"] = df["returns"].rolling(20, min_periods=8).std().fillna(0.0)

df[["date", "price", "returns", "rolling_vol_20"]].tail(12)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

axes[0].plot(df["date"], df["price"], color="#1f77b4", linewidth=1.8, label="Price")
axes[0].set_title("Commodity Price Level")
axes[0].set_xlabel("Date")
axes[0].set_ylabel("Price (USD)")
axes[0].legend()

axes[1].plot(df["date"], df["rolling_vol_20"] * 100, color="#d62728", linewidth=1.8, label="Rolling vol (20)")
axes[1].set_title("20-Period Realized Volatility")
axes[1].set_xlabel("Date")
axes[1].set_ylabel("Volatility (%)")
axes[1].legend()

plt.tight_layout()
plt.show()

## Interpretation and caveats

- Price trends and volatility trends can diverge; both are needed for risk context.
- Rolling volatility is sample-dependent and should be compared across fixed windows.

## Community extension ideas

- Add downside-only volatility and compare with full volatility.
- Segment by commodity family for cross-market comparison.